In [7]:
import requests
import json
from bs4 import BeautifulSoup
import re
import os
import subprocess
from pathlib import Path

SESSIONID = "1417A6C9F703A5E5609E60EC4E041563"

In [8]:
courseid_001 = "63d3a60fe4b0f86ecf1be779"
courseid_002 = "63d3e0d3e4b09032e4cb167c"
COOKIES = {"SESSIONID":SESSIONID}

SELECTED_COURSE_ID = courseid_001

In [3]:
r = requests.get(f"https://https://aathishsivasubrahmanian.graphy.com/s/mycourses?type=active&lsb",cookies=COOKIES)
apkid = re.search(r'apkId = \"(.*)\", theme9',r.text).group(1)
apkid

'6435364353532306361613936386239353338532306361613936386239353338'

In [4]:

r = requests.get(f"https://https://aathishsivasubrahmanian.graphy.com/s/courses/{SELECTED_COURSE_ID}/take",cookies=COOKIES)
soup = BeautifulSoup(r.content)
mydivs = soup.find("div",{"data-title":"8086 Video Lectures"}).find_all("div", {"class": "courseSubItem"})
viddict = {}
for div in mydivs:
    viddict[div.attrs["data-title"]] = div.attrs['data-id']

In [9]:
completedvids = ("01","02","03","04","05","06","07","08","09","10","11", "12",'13',"14","15","16","17","18","19","31","32","33","34","35","36")

for vid in viddict:
    if vid.startswith(completedvids) or isinstance(viddict[vid],dict):
        continue
    r = requests.get(f"https://https://aathishsivasubrahmanian.graphy.com/s/courses/{SELECTED_COURSE_ID}/videos/{viddict[vid]}/get",cookies=COOKIES)
    viddict[vid] = r.json()
print()

{
    "01 | Introduction to Microprocessors": "63d3a82de4b0c49f98c818e8",
    "02 | Basics of 8086": "63d3a790e4b06b2166845acc",
    "03 | Pipelining": "63d3a753e4b06b2166845abd",
    "04 | Memory Banking": "63d3dba7e4b0c73f8c073e72",
    "05 | Memory Segmentation": "63d3a7a5e4b06b2166845ace",
    "06 | Architecture": "63d429e9e4b0133c159e0874",
    "07 | Flag Register": "63d3a760e4b06b2166845ac3",
    "08 | Addressing Modes": "63d429eae4b0133c159e0876",
    "09 | Push & Pop Instructions": "63d429bfe4b0133c159e0868",
    "10 | XLAT Instruction": "63d42bfbe4b027668f761d05",
    "11 | IN & OUT Instructions": "63d420bfe4b03377216a0813",
    "12 | Arithmetic Group": "63d3d6c6e4b010248a895104",
    "13 | DAA & DAS Instructions": "63d42bf7e4b027668f761d04",
    "14 | Logic Group": "63d429cbe4b0133c159e0869",
    "15 | Rotate & Shift Group": "63d420d6e4b03377216a0815",
    "16 | Branch Control Group": "63d42135e4b03377216a0846",
    "17 | String Instructions": "63d4211ce4b03377216a0835",
    

In [6]:
for vid in viddict:
    if not isinstance(viddict[vid],dict):
        continue
    try:
        viddirpath = Path(vid)
        os.mkdir(vid)
    except Exception as e:
        print(e)
        pass
    url = viddict[vid]["spayee:resource"]["spayee:streamUrl"]
    r = requests.get(url,cookies=COOKIES)
    with open(viddirpath/"index.m3u8","wb") as f:
        f.write(r.content)
    NEEDS_SEPARATE_AUDIO = ("hls_audio_.m3u8" in r.text)
    
    r = requests.get(url[:-10]+"hls_500k_.m3u8",cookies=COOKIES)
    with open(viddirpath/"hls_500k_.m3u8","w") as f:
        f.write(r.content.decode("utf-8").replace("k/timestamp","key_video.bin"))
    
    with open(viddirpath/"hls_500k_.m3u8","r") as f:
        with open(viddirpath/"key_video.bin","wb") as f2:
            r = requests.get(url[:-10]+"k/timestamp",cookies=COOKIES)
            timestampreturn = list(bytearray(r.content))
            bytestowrite = eval((subprocess.check_output(["node","decrypt_key.js", apkid, str(timestampreturn)])).decode("utf-8").strip())
            f2.write(bytes(bytestowrite))
        for line in f.readlines():
            if line.startswith("hls_500k"):
                r = requests.get(url[:-10]+line.strip(),cookies=COOKIES)
                with open(viddirpath/line.strip(),"wb") as f2:
                    f2.write(r.content)

    if NEEDS_SEPARATE_AUDIO:
        r = requests.get(url[:-10]+"hls_audio_.m3u8",cookies=COOKIES)
        with open(viddirpath/"hls_audio_.m3u8","w") as f:
            f.write(r.content.decode("utf-8").replace("k/timestamp","key_audio.bin"))
        with open(viddirpath/"hls_audio_.m3u8","r") as f:
            with open(viddirpath/"key_audio.bin","wb") as f2:
                r = requests.get(url[:-10]+"k/timestamp",cookies=COOKIES)
                timestampreturn = list(bytearray(r.content))
                bytestowrite = eval((subprocess.check_output(["node","decrypt_key.js", apkid, str(timestampreturn)])).decode("utf-8").strip())
                f2.write(bytes(bytestowrite))
            for line in f.readlines():
                if line.startswith("hls_audio"):
                    r = requests.get(url[:-10]+line.strip(),cookies=COOKIES)
                    with open(viddirpath/line.strip(),"wb") as f2:
                        f2.write(r.content)

    break

[Errno 17] File exists: '18 | Programmers Model'
b'..\x00\x0c\x8fid\x80H"\x94T\x1b\xfc\x1cH'
b'..\x00\x0c\x8fid\x80H"\x94T\x1b\xfc\x1cH'


In [ ]:
"ffmpeg -allowed_extensions ALL -i '/Users/aathishs/Projects/CTF/BAMPMC/18 | Programmers Model/index.m3u8' Sample.mp4"